In [12]:
# !pip install gradio
# !pip install sentence-transformers
# !pip install datasets
# !pip install torch
# !pip install gdown
!pip install faiss-cpu

In [13]:
import pandas as pd
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers import InputExample
from torch.utils.data import DataLoader
import torch
import gradio as gr
import re
import json
import os
import gdown

In [14]:
#Check to see that GPU is running
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Apr 10 06:56:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   52C    P8             17W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [15]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import os
qa_path = '/content/drive/My Drive/amazon_qa/qa_Home_and_Kitchen.json'

print("Exists:", os.path.exists(qa_path))
print("Size:", os.path.getsize(qa_path), "bytes")


Exists: True
Size: 66602356 bytes


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
with open(qa_path, 'r') as f:
    for i in range(5):
        print(f.readline())


{'questionType': 'open-ended', 'asin': '1567120709', 'answerTime': 'Aug 5, 2014', 'unixTime': 1407222000, 'question': "If I can't change the battery myself, where would I get it changed?", 'answer': 'Go to anyone that changes watch batteries. CVS , Riteaid or Radio Shack!'}

{'questionType': 'yes/no', 'asin': '1567120709', 'answerTime': 'Aug 4, 2014', 'unixTime': 1407135600, 'question': 'Will this spell corrector give options of possible words the operator is trying to spell?', 'answerType': 'Y', 'answer': 'yes a small list of possiblities'}

{'questionType': 'open-ended', 'asin': '1567120709', 'answerTime': 'Oct 18, 2013', 'unixTime': 1382079600, 'question': 'What does the Fanklin Spelling corrector operate with (batteries)', 'answer': 'Yes, comes with a battery already installed.'}

{'questionType': 'yes/no', 'asin': '1567120709', 'answerTime': 'Feb 27, 2014', 'unixTime': 1393488000, 'question': 'Will it correct my spelling on Face Book?', 'answerType': '?', 'answer': 'The franklin s

In [19]:
import ast

qa_path = '/content/drive/My Drive/amazon_qa/qa_Home_and_Kitchen.json'

# Read each line and parse using literal_eval to solve issues with parsing unexpected characters
with open(qa_path, 'r') as f:
    data = [ast.literal_eval(line.strip()) for line in f if line.strip()]

qa_df = pd.DataFrame(data)
qa_df.head()


,questionType,asin,answerTime,unixTime,question,answer,answerType
0,open-ended,1567120709,"Aug 5, 2014",1.407222e+09,"If I can't change the battery myself, where wo...",Go to anyone that changes watch batteries. CVS...,NaN
1,yes/no,1567120709,"Aug 4, 2014",1.407136e+09,Will this spell corrector give options of poss...,yes a small list of possiblities,Y
2,open-ended,1567120709,"Oct 18, 2013",1.382080e+09,What does the Fanklin Spelling corrector opera...,"Yes, comes with a battery already installed.",NaN
3,yes/no,1567120709,"Feb 27, 2014",1.393488e+09,Will it correct my spelling on Face Book?,The franklin speller only corrects the word th...,?
4,yes/no,1567120709,"Jan 16, 2015",1.421395e+09,"Will it help a senior citizen who can't speak,...",no cuz it will stop working after the first we...,?


In [20]:
qa_df.columns

Index(['questionType', 'asin', 'answerTime', 'unixTime', 'question', 'answer',
       'answerType'],
      dtype='object')

In [21]:

reviewsfileid = "14GcJAzyN2PFg2JuyzF0pRmxlMmimrz9o"
reviewsurl = f"https://drive.google.com/uc?export=download&id={reviewsfileid}"
reviewsfilename = "AmazonHomeKitchenReviews.csv"


gdown.download(reviewsurl, reviewsfilename, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?export=download&id=14GcJAzyN2PFg2JuyzF0pRmxlMmimrz9o
From (redirected): https://drive.google.com/uc?export=download&id=14GcJAzyN2PFg2JuyzF0pRmxlMmimrz9o&confirm=t&uuid=506a5519-5385-4152-81c9-85678c30a6cb
To: /content/AmazonHomeKitchenReviews.csv

  0%|          | 0.00/692M [00:00<?, ?B/s]
  0%|          | 2.62M/692M [00:00<00:32, 21.0MB/s]
  1%|▏         | 8.91M/692M [00:00<00:18, 37.4MB/s]
  2%|▏         | 13.1M/692M [00:00<00:26, 25.8MB/s]
  3%|▎         | 21.5M/692M [00:00<00:23, 28.6MB/s]
  5%|▌         | 37.7M/692M [00:00<00:11, 56.8MB/s]
  7%|▋         | 45.6M/692M [00:01<00:13, 48.0MB/s]
  8%|▊         | 55.1M/692M [00:01<00:11, 55.6MB/s]
  9%|▉         | 62.4M/692M [00:01<00:14, 43.0MB/s]
 10%|█         | 71.3M/692M [00:01<00:12, 51.4MB/s]
 11%|█▏        | 78.1M/692M [00:01<00:11, 52.0MB/s]
 13%|█▎        | 86.5M/692M [00:01<00:11, 51.9MB/s]
 14%|█▍        | 99.1M/692M [00:02<00:12, 47.6MB/s]
 17%|█▋        | 116M/692

'AmazonHomeKitchenReviews.csv'

In [22]:
# read the reviews df
reviews_df = pd.read_csv(reviewsfilename,index_col=0 )
reviews_df.head()

,rating,title_x,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,title_y,description,price,Brand,Material,Color,categories
59,5,Adorable!,These are so sweet. I do wish the stopper part...,[],B01HBWGU80,B01DR2ACA0,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2019-07-23 04:29:16.671,0,True,"Little Bird Wine Bottle Stopper, Silicone Stop...",[],9.49,LouisChoice,Silicone,Assorted Color,"['Home & Kitchen', 'Kitchen & Dining', 'Kitche..."
87,5,"Stailess, healthier than coated pans","Great little stainless steel, balanced, good w...",[],B07T5CRVKQ,B08C7JYKZH,AEVWAM3YWN5URJVJIZZ6XPD2MKIA,2020-11-02 22:09:44.073,1,True,"Fortune Candy 8-Inch Fry Pan with Lid, 3-ply S...",[],24.99,Fortune Candy,"Stainless Steel, Aluminum",Mirror Finish,"['Home & Kitchen', 'Kitchen & Dining', 'Cookwa..."
89,5,Pretty colors available,Nice thin placemats of good size. Can be used ...,[],B07JRGZG6F,B0BYYYQ354,AEVWAM3YWN5URJVJIZZ6XPD2MKIA,2019-04-23 18:20:11.716,0,True,SHACOS Round Braided Placemats Set of 6 Round ...,[],16.99,SHACOS,Polyester,Pink,"['Home & Kitchen', 'Kitchen & Dining', 'Kitche..."
90,4,Nice material,"Very pretty, wish they came bigger",[],B00TW2M6YA,B01N1411P6,AEVWAM3YWN5URJVJIZZ6XPD2MKIA,2019-04-23 18:17:55.301,0,True,PaperLanternStore.com 7 Inch Bloom Shaped Hand...,"['This is our handmade 7"" Bloom shaped crochet...",6.92,Quasimoon,Cotton,Bloom - Beige/Ivory,"['Home & Kitchen', 'Kitchen & Dining', 'Kitche..."
93,4,Love the zipper!,The red is a deeper red rather than a bright r...,[],B01N6C4XJ7,B07BL41FQX,AEVWAM3YWN5URJVJIZZ6XPD2MKIA,2018-11-18 01:28:18.503,1,True,YANIBEST Silk Pillowcase for Hair and Skin 22 ...,[],29.99,YANIBEST,Silk,Ab-grey,"['Home & Kitchen', 'Bedding', 'Sheets & Pillow..."


In [23]:
reviews_df.columns

Index(['rating', 'title_x', 'text', 'images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase', 'title_y',
       'description', 'price', 'Brand', 'Material', 'Color', 'categories'],
      dtype='object')

In [24]:
#save in the amazon_qa dir in google drive
reviews_df.to_csv("/content/drive/My Drive/amazon_qa/AmazonHomeKitchenReviews.csv")

In [25]:
review_qa_df = pd.merge(qa_df, reviews_df, on="asin", how="inner")

print(f"There are {len(review_qa_df)} rows in the merged dataframe")
review_qa_df.head()

There are 301097 rows in the merged dataframe


,questionType,asin,answerTime,unixTime,question,answer,answerType,rating,title_x,text,...,timestamp,helpful_vote,verified_purchase,title_y,description,price,Brand,Material,Color,categories
0,open-ended,B00000JGRT,"Mar 26, 2011",1.301123e+09,berry frozen yogurt is hard as a rock! Any ide...,It has to do with sugar content. The higher th...,NaN,5,This Ice Cream Maker is a Gem,"It really is. Firstly, I have to admit that I...",...,2001-02-16 14:44:46.000,27,False,Cuisinart ICE-21PK Frozen Yogurt - Ice Cream &...,['The Cuisinart ICE-21PK automatic frozen yogu...,69.95,Cuisinart,Plastic,Pink,"['Home & Kitchen', 'Kitchen & Dining', 'Small ..."
1,open-ended,B00000JGRT,"Mar 26, 2011",1.301123e+09,berry frozen yogurt is hard as a rock! Any ide...,It has to do with sugar content. The higher th...,NaN,5,Ice Cream keeps us warm in winter?,"After several failed attempts, I began to wond...",...,2000-09-28 13:10:52.000,145,False,Cuisinart ICE-21PK Frozen Yogurt - Ice Cream &...,['The Cuisinart ICE-21PK automatic frozen yogu...,69.95,Cuisinart,Plastic,Pink,"['Home & Kitchen', 'Kitchen & Dining', 'Small ..."
2,open-ended,B00000JGRT,"Mar 26, 2011",1.301123e+09,berry frozen yogurt is hard as a rock! Any ide...,It has to do with sugar content. The higher th...,NaN,5,"Terrific, painless little ice cream maker!",I used to own a White Mountain hand crank ice ...,...,2005-08-06 15:02:01.000,6,False,Cuisinart ICE-21PK Frozen Yogurt - Ice Cream &...,['The Cuisinart ICE-21PK automatic frozen yogu...,69.95,Cuisinart,Plastic,Pink,"['Home & Kitchen', 'Kitchen & Dining', 'Small ..."
3,open-ended,B00000JGRT,"Mar 26, 2011",1.301123e+09,berry frozen yogurt is hard as a rock! Any ide...,It has to do with sugar content. The higher th...,NaN,5,Throw away the ice and salt. The new way to ma...,"I love homemade ice cream, but have not had lu...",...,2003-08-20 13:07:35.000,0,False,Cuisinart ICE-21PK Frozen Yogurt - Ice Cream &...,['The Cuisinart ICE-21PK automatic frozen yogu...,69.95,Cuisinart,Plastic,Pink,"['Home & Kitchen', 'Kitchen & Dining', 'Small ..."
4,open-ended,B00000JGRT,"Mar 26, 2011",1.301123e+09,berry frozen yogurt is hard as a rock! Any ide...,It has to do with sugar content. The higher th...,NaN,5,Love it,"This product is just fantastic. No mess, no co...",...,2010-07-01 09:51:01.000,0,True,Cuisinart ICE-21PK Frozen Yogurt - Ice Cream &...,['The Cuisinart ICE-21PK automatic frozen yogu...,69.95,Cuisinart,Plastic,Pink,"['Home & Kitchen', 'Kitchen & Dining', 'Small ..."


In [26]:
#rename title_y column to product title
review_qa_df.rename(columns={'asin': 'product_id', 'title_y': 'product_title', 'text':'review_text','title_x':'review_title'}, inplace=True)
review_qa_df.head()

,questionType,product_id,answerTime,unixTime,question,answer,answerType,rating,review_title,review_text,...,timestamp,helpful_vote,verified_purchase,product_title,description,price,Brand,Material,Color,categories
0,open-ended,B00000JGRT,"Mar 26, 2011",1.301123e+09,berry frozen yogurt is hard as a rock! Any ide...,It has to do with sugar content. The higher th...,NaN,5,This Ice Cream Maker is a Gem,"It really is. Firstly, I have to admit that I...",...,2001-02-16 14:44:46.000,27,False,Cuisinart ICE-21PK Frozen Yogurt - Ice Cream &...,['The Cuisinart ICE-21PK automatic frozen yogu...,69.95,Cuisinart,Plastic,Pink,"['Home & Kitchen', 'Kitchen & Dining', 'Small ..."
1,open-ended,B00000JGRT,"Mar 26, 2011",1.301123e+09,berry frozen yogurt is hard as a rock! Any ide...,It has to do with sugar content. The higher th...,NaN,5,Ice Cream keeps us warm in winter?,"After several failed attempts, I began to wond...",...,2000-09-28 13:10:52.000,145,False,Cuisinart ICE-21PK Frozen Yogurt - Ice Cream &...,['The Cuisinart ICE-21PK automatic frozen yogu...,69.95,Cuisinart,Plastic,Pink,"['Home & Kitchen', 'Kitchen & Dining', 'Small ..."
2,open-ended,B00000JGRT,"Mar 26, 2011",1.301123e+09,berry frozen yogurt is hard as a rock! Any ide...,It has to do with sugar content. The higher th...,NaN,5,"Terrific, painless little ice cream maker!",I used to own a White Mountain hand crank ice ...,...,2005-08-06 15:02:01.000,6,False,Cuisinart ICE-21PK Frozen Yogurt - Ice Cream &...,['The Cuisinart ICE-21PK automatic frozen yogu...,69.95,Cuisinart,Plastic,Pink,"['Home & Kitchen', 'Kitchen & Dining', 'Small ..."
3,open-ended,B00000JGRT,"Mar 26, 2011",1.301123e+09,berry frozen yogurt is hard as a rock! Any ide...,It has to do with sugar content. The higher th...,NaN,5,Throw away the ice and salt. The new way to ma...,"I love homemade ice cream, but have not had lu...",...,2003-08-20 13:07:35.000,0,False,Cuisinart ICE-21PK Frozen Yogurt - Ice Cream &...,['The Cuisinart ICE-21PK automatic frozen yogu...,69.95,Cuisinart,Plastic,Pink,"['Home & Kitchen', 'Kitchen & Dining', 'Small ..."
4,open-ended,B00000JGRT,"Mar 26, 2011",1.301123e+09,berry frozen yogurt is hard as a rock! Any ide...,It has to do with sugar content. The higher th...,NaN,5,Love it,"This product is just fantastic. No mess, no co...",...,2010-07-01 09:51:01.000,0,True,Cuisinart ICE-21PK Frozen Yogurt - Ice Cream &...,['The Cuisinart ICE-21PK automatic frozen yogu...,69.95,Cuisinart,Plastic,Pink,"['Home & Kitchen', 'Kitchen & Dining', 'Small ..."


In [27]:
print(f"Columns after merge and cleanup: {review_qa_df.columns}")

review_qa_df = review_qa_df[['questionType', 'product_id', 'review_text', 'product_title','rating', 'question', 'answer', 'rating', 'price',  'Color', 'Material']]
review_qa_df.head()

Columns after merge and cleanup: Index(['questionType', 'product_id', 'answerTime', 'unixTime', 'question',
       'answer', 'answerType', 'rating', 'review_title', 'review_text',
       'images', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote',
       'verified_purchase', 'product_title', 'description', 'price', 'Brand',
       'Material', 'Color', 'categories'],
      dtype='object')


,questionType,product_id,review_text,product_title,rating,question,answer,rating,price,Color,Material
0,open-ended,B00000JGRT,"It really is. Firstly, I have to admit that I...",Cuisinart ICE-21PK Frozen Yogurt - Ice Cream &...,5,berry frozen yogurt is hard as a rock! Any ide...,It has to do with sugar content. The higher th...,5,69.95,Pink,Plastic
1,open-ended,B00000JGRT,"After several failed attempts, I began to wond...",Cuisinart ICE-21PK Frozen Yogurt - Ice Cream &...,5,berry frozen yogurt is hard as a rock! Any ide...,It has to do with sugar content. The higher th...,5,69.95,Pink,Plastic
2,open-ended,B00000JGRT,I used to own a White Mountain hand crank ice ...,Cuisinart ICE-21PK Frozen Yogurt - Ice Cream &...,5,berry frozen yogurt is hard as a rock! Any ide...,It has to do with sugar content. The higher th...,5,69.95,Pink,Plastic
3,open-ended,B00000JGRT,"I love homemade ice cream, but have not had lu...",Cuisinart ICE-21PK Frozen Yogurt - Ice Cream &...,5,berry frozen yogurt is hard as a rock! Any ide...,It has to do with sugar content. The higher th...,5,69.95,Pink,Plastic
4,open-ended,B00000JGRT,"This product is just fantastic. No mess, no co...",Cuisinart ICE-21PK Frozen Yogurt - Ice Cream &...,5,berry frozen yogurt is hard as a rock! Any ide...,It has to do with sugar content. The higher th...,5,69.95,Pink,Plastic


In [28]:
review_qa_df.columns

Index(['questionType', 'product_id', 'review_text', 'product_title', 'rating',
       'question', 'answer', 'rating', 'price', 'Color', 'Material'],
      dtype='object')

In [29]:
review_qa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301097 entries, 0 to 301096
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   questionType   301097 non-null  object 
 1   product_id     301097 non-null  object 
 2   review_text    301057 non-null  object 
 3   product_title  301097 non-null  object 
 4   rating         301097 non-null  int64  
 5   question       301097 non-null  object 
 6   answer         301097 non-null  object 
 7   rating         301097 non-null  int64  
 8   price          301097 non-null  float64
 9   Color          301097 non-null  object 
 10  Material       301097 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 25.3+ MB


### SET UP TRAINING EXAMPLES

In [30]:

# Initialize model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Prepare examples (both positive and negative)
train_examples = []
for product_id, group in review_qa_df.groupby("product_id"):
    for _, row in group.iterrows():
      product_name = row['product_title']
      rating = row['rating']
      # Positive pair: (question, answer)
      question = f"{product_name}: row['question']"
      answer = f"({product_name}, Rating: {rating}): row['answer']"

      # Add positive example
      train_examples.append(InputExample(texts=[question, answer]))

      other_answers = group[group['answer'] != row['answer']]
      #Check to see if the data frame is empty. If not, proceed with negative sampling
      if not other_answers.empty:
          negative_answer = other_answers.sample(1)['answer'].values[0]
          negative_answer = f"({product_name}, Rating:{rating/5}): {negative_answer}"
          train_examples.append(InputExample(texts=[question, negative_answer]))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### EVALUATION METRIC

In [39]:
def save_model_checkpoint(model, output_path, epoch):
    checkpoint_path = os.path.join(output_path, f"checkpoint_epoch_{epoch}")
    model.save(checkpoint_path)
    print(f"Checkpoint saved at epoch {epoch} to {checkpoint_path}")

def load_model_checkpoint(model_name, checkpoint_path):
    model = SentenceTransformer(model_name)
    model.load(checkpoint_path)
    print(f"Model loaded from checkpoint {checkpoint_path}")
    return model


checkpoint_path = "/content/checkpoints"

# Check if a checkpoint exists in the output_path
if os.path.exists(checkpoint_path):
    print(f"Resuming training from checkpoint: {checkpoint_path}")
    model = SentenceTransformer(checkpoint_path)  # Load model from checkpoint folder
else:
    print("No checkpoint found, starting fresh.")
    model = SentenceTransformer(model_name)  # Load from the base model if no checkpoint exists

In [31]:
def evaluate_model(model, validation_split):
    """
    Evaluate model using information retrieval metrics like MRR, MAP, etc.
    """
    # Create validation queries and answers
    queries = []
    relevant_answers = []

    for example in validation_split:
        query = example.texts[0]
        relevant_answer = example.texts[1]

        queries.append(query)
        relevant_answers.append(relevant_answer)

    # Create embeddings for all answers (corpus)
    corpus = [ex.texts[1] for ex in validation_split]
    corpus_embeddings = model.encode(corpus, convert_to_tensor=False)

    # Encode queries
    query_embeddings = model.encode(queries, convert_to_tensor=False)

    # Compute Mean Reciprocal Rank (MRR)
    mrr_sum = 0
    for i, query_embedding in enumerate(query_embeddings):
        # Compute cosine similarity
        similarities = cosine_similarity([query_embedding], corpus_embeddings)[0]

        # Get ranking of relevant answer
        relevant_idx = corpus.index(relevant_answers[i])
        ranked_indices = np.argsort(similarities)[::-1]
        rank = np.where(ranked_indices == relevant_idx)[0][0] + 1

        # Add reciprocal rank
        mrr_sum += 1.0 / rank

    mrr = mrr_sum / len(queries)
    return mrr

In [ ]:
def save_model_checkpoint(model, output_path, epoch):
    checkpoint_path = os.path.join(output_path, f"checkpoint_epoch_{epoch}")
    model.save(checkpoint_path)
    print(f"Checkpoint saved at epoch {epoch} to {checkpoint_path}")

def load_model_checkpoint(model_name, checkpoint_path):
    model = SentenceTransformer(model_name)
    model.load(checkpoint_path)
    print(f"Model loaded from checkpoint {checkpoint_path}")
    return model

In [42]:
#train with different loss functions to find the loss that optimizes the model
from sklearn.model_selection import train_test_split


model_name ='all-MiniLM-L6-v2'

def evaluate_model_loss_functions(loss_functions=None, model_name=model_name, epochs=1, batch_size=16, val_split=0.2, warmup_steps=100, checkpoint_epoch=0):
    """
    Train and evaluate models with multiple loss functions

    Parameters:
    - loss_functions: List of loss function classes or None to use defaults
    - model_name: Base model to use
    - epochs: Number of training epochs
    - batch_size: Batch size for training
    - warmup_steps: Number of warmup steps

    Returns:
    - Dictionary mapping loss function names to (model, validation_score)
    """
    # If no loss functions provided, use these defaults
    if loss_functions is None:
        from sentence_transformers import losses
        loss_functions = [
            losses.MultipleNegativesRankingLoss,
            losses.TripletLoss,
            losses.ContrastiveLoss,
            losses.CosineSimilarityLoss,
            losses.BatchHardTripletLoss,
            losses.OnlineContrastiveLoss
        ]

    results = {}

    # Create train/validation split
    if val_split > 0:
        train_data, validation_data = train_test_split(
            train_examples,
            test_size=val_split,
            random_state=42
        )

    # (You'd need to implement the train/validation split logic)


    for loss_class in loss_functions:
        loss_name = loss_class.__name__
        print(f"Training with {loss_name}...")

        # Check if a checkpoint exists to resume training from
        output_path = f'/content/drive/My Drive/amazon_qa/amazon-qa-model-{loss_name}'
        checkpoint_path = os.path.join(output_path, f"checkpoint_epoch_{checkpoint_epoch}")

        # If the checkpoint exists, resume training from there
        if os.path.exists(checkpoint_path):
            print(f"Resuming training from checkpoint: {checkpoint_path}")
            model = SentenceTransformer(output_path)  # Load the model from the checkpoint folder
        else:
            print("No checkpoint found, starting fresh.")
            model = SentenceTransformer(model_name)

        # Initialize the loss function
        if loss_class == losses.TripletLoss:
            loss_function = loss_class(model, triplet_margin=1.0)
        elif loss_class in [losses.CosineSimilarityLoss, losses.ContrastiveLoss,
                           losses.BatchHardTripletLoss, losses.OnlineContrastiveLoss]:
            loss_function = loss_class(model)
        else:
          # MultipleNegativesRankingLoss and others
            loss_function = loss_class(model)

        # Create a new dataloader
        train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

        # Train the model
        output_path = f'/content/drive/My Drive/amazon_qa/amazon-qa-model-{loss_name}'
        model.fit(
            train_objectives=[(train_dataloader, loss_function)],
            epochs=epochs,
            warmup_steps=warmup_steps,
            show_progress_bar=True,
            output_path=output_path
        )

        # Evaluate the model
        val_score = evaluate_model(model, validation_data)
        print(f"{loss_name} validation score: {val_score:.4f}")

        # Store results
        results[loss_name] = (model, val_score)

    # Print summary of results
    print("\nSummary of results:")
    for loss_name, (_, score) in sorted(results.items(), key=lambda x: x[1][1], reverse=True):
        print(f"{loss_name}: {score:.4f}")

    # Return the best model based on validation score
    best_loss = max(results.items(), key=lambda x: x[1][1])[0]
    print(f"\nBest loss function: {best_loss}")

    return results



In [37]:

# Save the fine-tuned model
#model.save('/content/drive/My Drive/amazon_qa/MiniLM_fine_tuned_model')

In [ ]:

# Run evaluation with default loss functions
results = evaluate_model_loss_functions(
    loss_functions=None,
    model_name=model_name,
    epochs=3,
    batch_size=16,
    val_split=0.2,
    warmup_steps=100
)

# Get the best model
best_loss_name = max(results.items(), key=lambda x: x[1][1])[0]
best_model, best_score = results[best_loss_name]

# Save the best model
best_model.save(f'/content/drive/My Drive/amazon_qa/amazon-qa-best-model-{best_loss_name}')

Training with MultipleNegativesRankingLoss...
No checkpoint found, starting fresh.


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,0.309500
1000,0.296200
1500,0.313400
2000,0.300900
2500,0.295100
3000,0.300100
3500,0.274500
4000,0.300000
4500,0.305700
5000,0.296700


Step,Training Loss
500,0.309500
1000,0.296200
1500,0.313400
2000,0.300900
2500,0.295100
3000,0.300100
3500,0.274500
4000,0.300000
4500,0.305700
5000,0.296700


## CREATE SEARCH INDEX USING VECTOR EMBEDDINGS

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import faiss

# Create embeddings for all your answers
answer_texts = []
answer_metadata = []

for _, row in review_qa_df.iterrows():
    product_name = row['product_title']
    rating = row['rating']
    answer = f"({product_name}, {rating}): {row['answer']}"
    answer_texts.append(answer)
    answer_metadata.append({
        'product_id': row['product_id'],
        'question': row['question'],
        'rating': rating,
        'product_name': product_name
    })

# Generate embeddings for all answers
answer_embeddings = model.encode(answer_texts, convert_to_tensor=False, show_progress_bar=True)

# Use FAISS for faster similarity search with large datasets
dimension = answer_embeddings.shape[1]
# Inner product (cosine on normalized vectors)
faiss_index_st = faiss.IndexFlatIP(dimension)
# Normalize vectors for cosine similarity
faiss.normalize_L2(answer_embeddings)
faiss_index_st.add(answer_embeddings)

In [ ]:
#save faiss index to disk and to google drive
faiss.write_index(faiss_index_st, "my_faiss_index.bin")
faiss.write_index(faiss_index_st, "/content/drive/My Drive/vector2/my_faiss_index.bin")


#### Faiss retrun the score based on the type of index created, we created faiss1D which uses the inner product

*   IndexFlatIP - Inner product similarity (higher is better)
*   IndexFlatL2 - L2 distance/Euclidean distance (lower is better)
*   IndexFlatCosine - Cosine similarity (higher is better)




### IMPLEMENT SEMANTIC SEARCH FUNCTION TO CALL IN STREAMLIT UI

In [ ]:
def semantic_search(query, top_k=5):
    # Encode the query
    query_embedding = model.encode(query, convert_to_tensor=False)
    query_embedding = query_embedding.reshape(1, -1)

    # Normalize for cosine similarity
    query_embedding = query_embedding / np.linalg.norm(query_embedding)

    # Using FAISS vectorstore
    D, I = faiss_index_st.search(query_embedding, top_k)
    results = []
    for i, idx in enumerate(I[0]):
       # FAISS returns -1 for not enough results
        if idx != -1:
            results.append({
                'answer': answer_texts[idx],
                'score': float(D[0][i]),
                'metadata': answer_metadata[idx]
            })

    return results



### EVALUATE THE MODEL

In [ ]:
# Create a test set
test_queries = ["How durable is this Cuisinart 21K ice cream like?", "what is the highest rated ice cream maker?"]

# Run some test searches
for query in test_queries:
    print(f"Query: {query}")
    results = semantic_search(query, top_k=2)
    for rank, result in enumerate(results, 1):
        print(f"Rank {rank}: {result['answer']} (Score: {result['score']:.4f})")
    print()


In [45]:
#export final review_qa_df to drive
review_qa_df.to_csv("/content/drive/My Drive/amazon_qa/review_qa_df.csv")